# Business Case 3

Group members:
- Lorenzo Pigozzi	--- m20200745
- Nguyen Huy Phuc	--- m20200566
- Ema Mandura	  --- m20200647


## Market Basket Analysis

Objectives


- What are the main types of consumer behavior in the business?
- Which types of products should have an extended amount of product offerings?
- Which types of products can be seen as substitutes?
- Which items are complementary?

<a class="anchor" id="0.1"></a>
# **Table of Contents**

1.	[Importing data and libraries](#1)   
2.	[Exploratory data analysis (EDA)](#2)       

# 1. Importing data and libraries <a class="anchor" id="1"></a>

In [110]:
# Import packages
import pandas as pd
import numpy as np
import datetime as dt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

In [111]:
# importing the 4 datasets
products = pd.read_csv('datasets/products.csv')
departments = pd.read_csv('datasets/departments.csv')
orders = pd.read_csv('datasets/orders.csv')
order_products = pd.read_csv('datasets/order_products.csv')

In [112]:
# renaming a column in 'departments'
departments.columns = ['department_id', 'department_name']

# merging the different datasets in one
product_departments = pd.merge(products, departments, how='left', on='department_id').\
                        drop(["department_id"], axis=1)

order_product_departments = pd.merge(order_products, product_departments, how='left', on='product_id').\
                        drop(["product_id"], axis=1)

df = pd.merge(order_product_departments, orders, how='left', on='order_id')

# 2. EDA <a class="anchor" id="2"></a>

In [113]:
df.head(5)

,order_id,add_to_cart_order,reordered,product_name,department_name,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,10,1,1,fresh fruits,produce,135442,4,6,8,8.0
1,10,2,1,fresh vegetables,produce,135442,4,6,8,8.0
2,10,3,0,fresh herbs,produce,135442,4,6,8,8.0
3,10,4,1,fresh fruits,produce,135442,4,6,8,8.0
4,10,5,1,fresh vegetables,produce,135442,4,6,8,8.0


In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2019501 entries, 0 to 2019500
Data columns (total 10 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   add_to_cart_order       int64  
 2   reordered               int64  
 3   product_name            object 
 4   department_name         object 
 5   user_id                 int64  
 6   order_number            int64  
 7   order_dow               int64  
 8   order_hour_of_day       int64  
 9   days_since_prior_order  float64
dtypes: float64(1), int64(7), object(2)
memory usage: 169.5+ MB


In [115]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
order_id,2.0195e+06,NaN,NaN,NaN,1.70701e+06,985983,10,852649,1.705e+06,2.55903e+06,3.42108e+06
add_to_cart_order,2.0195e+06,NaN,NaN,NaN,8.36317,7.15006,1,3,6,11,137
reordered,2.0195e+06,NaN,NaN,NaN,0.589743,0.49188,0,0,1,1,1
product_name,2019501,134,fresh fruits,226039,NaN,NaN,NaN,NaN,NaN,NaN,NaN
department_name,2019501,21,produce,588996,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_id,2.0195e+06,NaN,NaN,NaN,103067,59491.2,2,51584,102690,154600,206209
order_number,2.0195e+06,NaN,NaN,NaN,17.1514,17.5258,1,5,11,24,100
order_dow,2.0195e+06,NaN,NaN,NaN,2.73537,2.09388,0,1,3,5,6
order_hour_of_day,2.0195e+06,NaN,NaN,NaN,13.4395,4.24101,0,10,13,16,23
days_since_prior_order,1.89516e+06,NaN,NaN,NaN,11.386,8.97098,0,5,8,15,30


In [116]:
df.isna().sum()

order_id                       0
add_to_cart_order              0
reordered                      0
product_name                   0
department_name                0
user_id                        0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    124342
dtype: int64

In [117]:
df.shape

(2019501, 10)

In [118]:
order_products.shape

(2019501, 4)

In [119]:
orders.shape

(200000, 6)

First of all, it's important to keep in mind what each row of the dataset represents. Indeed, the id is a combination of the product_id and the order_id.\
Thus, in the dataset are stored 2019501 products sold.

In [120]:
df['order_id'].value_counts()

790903     137
2621625    109
416346     100
1031566     95
1930316     93
          ... 
1656968      1
1011835      1
2220793      1
9773         1
3084549      1
Name: order_id, Length: 200000, dtype: int64

In [121]:
df['user_id'].value_counts()

176478    460
129928    405
126305    384
201268    347
115495    283
         ... 
52650       1
166234      1
42637       1
81520       1
57078       1
Name: user_id, Length: 105273, dtype: int64

In [122]:
print('Summary')
print('--------------------------------------------------------')
print('Total number of products bought : ', len(df))
print('Total number of orders : ', len(df['order_id'].unique()))
print('Total number of customers : ', len(df['user_id'].unique()))
print('---------------------------------------------------------')
print('Number of customer with more than one order : ',
    len(df['order_id'].unique()) - len(df['user_id'].unique()))
print('Average number of products for each order : ', 
      len(df) / len(df['order_id'].unique()))

Summary
--------------------------------------------------------
Total number of products bought :  2019501
Total number of orders :  200000
Total number of customers :  105273
---------------------------------------------------------
Number of customer with more than one order :  94727
Average number of products for each order :  10.097505


Notes:
- Total number of products bought: 2019501
- Total number of orders:  200000
- Total number of customers: 105273

- Number of customers whith more than one order: 94727
- Average number of products for each order : 10

In [123]:
orders['days_since_prior_order'].isna().sum()

12254

## Analysis of the product offerings
Question : Which types of products should have an extended amount of product offerings?

In [124]:
# checking the top products (most sold)
top_products = pd.DataFrame(df['product_name'].value_counts().head(10)).reset_index()
top_products.columns = ['product_name', 'value']

fig = px.bar(top_products, y = 'product_name', x = 'value', orientation='h', 
             color="product_name", color_discrete_sequence=px.colors.qualitative.Antique,
            title='Top products for selling')
fig.show()

In [165]:
# transformation of the variable day of the week
# changing the name
df_for_plot = df.copy()
df_for_plot['day_of_week'] = df_for_plot['order_dow']
df_for_plot.drop('order_dow', axis=1, inplace=True)
# changing the values
df_for_plot['day_of_week'] = df_for_plot['day_of_week'].replace([0,1,2,3,4,5,6],  
                    ['Sunday','Monday', 'Tuesday', 'Wednesday','Thursday','Friday', 'Saturday'])

# checking the top products (most sold)
top_dow = pd.DataFrame(df_for_plot['day_of_week'].value_counts().head(10)).reset_index()
top_dow.columns = ['day_of_week', 'value']
fig = px.bar(top_dow, y = 'day_of_week', x = 'value', orientation='h', 
            color_discrete_sequence=px.colors.qualitative.Antique, color="day_of_week", 
            title='Top day of the week for selling')
fig.show()

In [141]:
hour_frequency = df['order_hour_of_day'].value_counts().reset_index()

hour_frequency.columns = ['hour', 'total_frequency']

hour_frequency = hour_frequency.sort_values(by='hour')

##plotting##
data_for_plot = dict(type='scatter',
                x=hour_frequency['hour'],
                y=hour_frequency['total_frequency'])
# setting the layout
plot_1_layout = dict(title=dict(text='Frequency purchases per hour of the day'),
                      xaxis=dict(title='Hour'),
                      yaxis=dict(title='Total Frequency')
                      )
# displaying the graph
plot_1 = go.Figure(data=data_for_plot, layout=plot_1_layout)
# plot_1.show(renderer='png')
plot_1.show()

In [133]:
df

,order_id,add_to_cart_order,reordered,product_name,department_name,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,day_of_week
0,10,1,1,fresh fruits,produce,135442,4,6,8,8.0,6
1,10,2,1,fresh vegetables,produce,135442,4,6,8,8.0,6
2,10,3,0,fresh herbs,produce,135442,4,6,8,8.0,6
3,10,4,1,fresh fruits,produce,135442,4,6,8,8.0,6
4,10,5,1,fresh vegetables,produce,135442,4,6,8,8.0,6
...,...,...,...,...,...,...,...,...,...,...,...
2019496,3420578,17,0,asian foods,international,6586,6,5,21,5.0,5
2019497,3420578,18,1,fresh herbs,produce,6586,6,5,21,5.0,5
2019498,3420693,1,0,ice cream ice,frozen,68287,15,6,15,6.0,6
2019499,3420693,2,0,ice cream ice,frozen,68287,15,6,15,6.0,6


In [149]:
df_for_plot = df.groupby(['product_name', 'order_hour_of_day']).size()

df_for_plot = pd.DataFrame(df_for_plot).reset_index()

df_for_plot.columns = ['product', 'hour_of_day', 'frequency']


# manipulating the data with pivot table
df_for_plot = pd.pivot_table(df_for_plot, values=['frequency'],columns=['product'], index = ['hour_of_day'])
# dropping the multi-index level for the columns
df_for_plot.columns = df_for_plot.columns.droplevel(0)

##plotting##
data_for_plot = [dict(type='scatter',
                x=df_for_plot.index,
                y=df_for_plot[product],
                name=product)
                 for product in df_for_plot.columns
                ]
    
# setting the layout
plot_1_layout = dict(title=dict(text='Frequency purchases per hour of the day'),
                      xaxis=dict(title='Hour of the day'),
                      yaxis=dict(title='Total Frequency')
                      )
# displaying the graph
plot_1 = go.Figure(data=data_for_plot, layout=plot_1_layout)
# plot_1.show(renderer='png')
plot_1.show()

In [147]:
df_for_plot = df.groupby(['product_name', 'day_of_week']).size()

df_for_plot = pd.DataFrame(df_for_plot).reset_index()

df_for_plot.columns = ['product', 'day_of_week', 'frequency']

# manipulating the data with pivot table
df_for_plot = pd.pivot_table(df_for_plot, values=['frequency'],columns=['product'], index = ['day_of_week'])
# dropping the multi-index level for the columns
df_for_plot.columns = df_for_plot.columns.droplevel(0)

labels_plot = {}
##plotting##
data_for_plot = [dict(type='scatter',
                x=df_for_plot.index,
                y=df_for_plot[product],
                name=product)
                 for product in df_for_plot.columns
                ]

# setting the layout
plot_1_layout = dict(title=dict(text='Frequency purchases per day of the week'),
                      xaxis=dict(title='Day of the week'),
                      yaxis=dict(title='Total Frequency')
                      )
# displaying the graph
plot_1 = go.Figure(data=data_for_plot, layout=plot_1_layout)
# plot_1.show(renderer='png')
plot_1.show()

In [162]:
def product_per_hour(product):
    df_for_plot = df.groupby(['product_name', 'order_hour_of_day']).size()

    df_for_plot = pd.DataFrame(df_for_plot).reset_index()

    df_for_plot.columns = ['product', 'hour_of_day', 'frequency']

    # manipulating the data with pivot table
    df_for_plot = pd.pivot_table(df_for_plot, values=['frequency'],columns=['product'], index = ['hour_of_day'])
    # dropping the multi-index level for the columns
    df_for_plot.columns = df_for_plot.columns.droplevel(0)

    ##plotting##
    data_for_plot = dict(type='scatter',
                    x=df_for_plot.index,
                    y=df_for_plot[product],
                    name=product)

    # setting the layout
    plot_1_layout = dict(title=dict(text='Frequency purchases per hour of the day'),
                          xaxis=dict(title='Hour of the day'),
                          yaxis=dict(title='Total Frequency')
                          )
    # displaying the graph
    plot_1 = go.Figure(data=data_for_plot, layout=plot_1_layout)
    
    plot_1.show()
    
    
    df_for_plot = df.groupby(['product_name', 'day_of_week']).size()
    df_for_plot = pd.DataFrame(df_for_plot).reset_index()
    df_for_plot.columns = ['product', 'day_of_week', 'frequency']
    # manipulating the data with pivot table
    df_for_plot = pd.pivot_table(df_for_plot, values=['frequency'],columns=['product'], index = ['day_of_week'])
    # dropping the multi-index level for the columns
    df_for_plot.columns = df_for_plot.columns.droplevel(0)

    labels_plot = {}
    ##plotting##
    data_for_plot = dict(type='scatter',
                    x=df_for_plot.index,
                    y=df_for_plot[product],
                    name=product)

    # setting the layout
    plot_2_layout = dict(title=dict(text='Frequency purchases per day of the week'),
                          xaxis=dict(title='Day of the week'),
                          yaxis=dict(title='Total Frequency')
                          )
    # displaying the graph
    plot_2 = go.Figure(data=data_for_plot, layout=plot_2_layout)
    # plot_1.show(renderer='png')
    plot_2.show()
    

In [163]:
product_per_hour('red wines')

In [168]:
df['product_name'].unique().tolist()

['fresh fruits',
 'fresh vegetables',
 'fresh herbs',
 'baby food formula',
 'canned meals beans',
 'cream',
 'meat counter',
 'frozen meals',
 'fresh dips tapenades',
 'chips pretzels',
 'oils vinegars',
 'butter',
 'other creams cheeses',
 'poultry counter',
 'milk',
 'spreads',
 'packaged cheese',
 'yogurt',
 'canned jarred vegetables',
 'eggs',
 'frozen produce',
 'doughs gelatins bake mixes',
 'packaged vegetables fruits',
 'refrigerated',
 'seafood counter',
 'soy lactosefree',
 'coffee',
 'water seltzer sparkling water',
 'paper goods',
 'frozen juice',
 'vitamins supplements',
 'soup broth bouillon',
 'energy granola bars',
 'soft drinks',
 'crackers',
 'spices seasonings',
 'baking ingredients',
 'asian foods',
 'cereal',
 'condiments',
 'hot dogs bacon sausage',
 'honeys syrups nectars',
 'canned fruit applesauce',
 'cleaning products',
 'oral hygiene',
 'bread',
 'packaged poultry',
 'instant foods',
 'dry pasta',
 'prepared soups salads',
 'granola',
 'popcorn jerky',
 'pro